In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df_test = pd.read_csv("test_data.csv")
df_train = pd.read_csv("train_data.csv")
df_test_prediction = pd.read_csv("test_data_prediction.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_test_prediction = df_test_prediction['v39']

In [5]:
def drop_columns(df):
    df = df.drop({"v1","v2","v4","v19"}, axis=1)
    
    return df

In [6]:
# Fixing categorical data 
def categorical_to_numerical(df):
    cleanup_nums =     {"v5": {"ASC" : 1, "DEALER" : 0},
                        "v11": {"P": 0, "A": 1, "G": 2},
                        "v12": {"P": 0, "A": 1, "G": 2},
                        "v13": {"P": 0, "A": 1, "G": 2},
                        "v14": {"P": 0, "A": 1, "G": 2},
                        "v37": {"N": 0, "Y": 1},
                        "v38": {"NO": 0, "YES": 1}
                        }
    df = df.replace(cleanup_nums)
    
    return df

In [7]:
def impute_columns(df_train_imputedValues):
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues["v10"].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v11'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v12'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v13'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v14'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v17'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v18'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v24'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v32'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v33'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v34'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v35'].mode()[0])
    df_train_imputedValues = df_train_imputedValues.fillna(df_train_imputedValues['v36'].mode()[0])
    
    return df_train_imputedValues

In [8]:
from scipy.stats import norm, skew 

def fix_skew(df):
    from scipy.special import boxcox1p
    skewed_features = ["v32","v34","v27","v24","v38","v33","v29","v10","v15","v7",]
    lam = 0.15
    for feat in skewed_features:
        #all_data[feat] += 1
        df[feat] = boxcox1p(df[feat], lam)
        
    return df

In [9]:
df_train = drop_columns(df_train)
df_train= categorical_to_numerical(df_train)
df_train = impute_columns(df_train)
df_train = fix_skew(df_train)

In [10]:
df_test = drop_columns(df_test)
df_test = categorical_to_numerical(df_test)
df_test = impute_columns(df_test)
df_test = fix_skew(df_test)

In [11]:
y_train = df_train['v39']
X_train = df_train.drop(columns='v39')
X_test = df_test
y_test = df_test_prediction

print('X train shape: ', X_train.shape)
print('Y train shape: ', y_train.shape)
print('X test shape: ', X_test.shape)
print('Y test shape: ', y_test.shape)


X train shape:  (115786, 34)
Y train shape:  (115786,)
X test shape:  (11263, 34)
Y test shape:  (11263,)


In [13]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred=xgb.predict(X_test)

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
# from xgboost import XGBClassifier

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.07], #so called `eta` value
              'max_depth': [9],
              'min_child_weight': [1],
              'silent': [1],
              'subsample': [1],
              'colsample_bytree': [0.5],
              'n_estimators': [1500]}

xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_) 

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[12:50:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[12:50:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


0.6268174327511986
{'colsample_bytree': 0.5, 'learning_rate': 0.07, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 1500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 1}


In [15]:
y_pred=xgb_grid.predict(X_test)

from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("\t\tError Table")
print('Mean Absolute Error      : ', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared  Error      : ', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared  Error : ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R Squared Error          : ', metrics.r2_score(y_test, y_pred))

		Error Table
Mean Absolute Error      :  5032.400503602087
Mean Squared  Error      :  45792484.38974433
Root Mean Squared  Error :  6767.0144369392565
R Squared Error          :  0.8099829016758902


In [17]:
import joblib

#save your model or results
joblib.dump(xgb_grid, 'model.pkl')
print("model saved")

#load your model for further usage
# joblib.load("model_file_name.pkl")

model saved
